#### Basics ####

In [ ]:
### Getting Help

    Get-Command | where CommandType -eq 'Cmdlet' | Format-Wide -Column 2
    Get-Command -Noun Process | Format-Wide -Column 2

    Get-Process -?
    Get-Help Get-Process -Online
    Get-Help Get-Process -ShowWindow
    Get-Help Get-Process -Full

#Powershell version
    $PSVersionTable

In [ ]:
### Output

    Format-Table
    Format-Wide
    Format-List
    Format-Custom
    Get-Service | Sort-Object Name | Format-List Name, Status, CanStop

    Out-Default
    Out-File
    Out-GridView
    Out-Host
    Out-Null
    Out-String

    Get-Process code | Out-GridView -Title "Процессы" -PassThru 
    Get-Process code | Format-Table Id, ProcessName

#### Debugging and logging ####

In [ ]:
# ErrorAction

    $ErrorActionPreference = "Inquire"
    #Continue (Default)
    #Stop

    Get-ChildItem C:\1.txt -ErrorAction Inquire

In [ ]:
# Debug

    Set-PSDebug -Trace 1 #1: Trace script lines as they run. 2: Trace script lines, variable assignments, function calls, and scripts.
    Set-PSDebug -Step #script steping
    Set-PSDebug -Off

In [ ]:
# Trace

    Get-TraceSource | Sort-Object Name | Format-Table -AutoSize

    Trace-Command -Name ParameterBinding, MemberResolution -Expression {Get-MailDomainInfo -DomainName powershellisfun.com} -PSHost

    Trace-Command -Name * -Expression {Get-Process} -PSHost

In [ ]:
#Verbose

    Copy-Item -Path "C:\source\example.txt" -Destination "C:\destination\" -Verbose
    #VERBOSE: Performing the operation "Copy File" on target "Item: C:\source\example.txt Destination: C:\destination\example.txt".

    <# Записывать данные в различные потоки можно с помощью соответствующих
    командлетов с глаголом Write: Write-Output, Write-Error, Write-Warning, Write-
    Verbose, Write-Debug И Write-Information.
    Например, запишем в поток ошибок новое сообщение: #>

    Write-Error -Message "Произошла ошибка" 

#### Working wih objects ####

In [ ]:
### Get-Member

    $process = Get-Process | Select-Object -First 1
    $process | Get-Member #instance methods
    $process | Get-Member -Static #class methods

    [System.Diagnostics.Process] | Get-Member -Static
    [math] | Get-Member -Static

In [ ]:
# Measure-Object

    dir | Measure-Object -Property Length -Sum #display sum of file sizes

In [ ]:
# Compare-Object

    $a = @{ "one" =  1; "two"= 2 }
    $b = @{ "one" =  1; "three" =  3; "two"= 22 }

    Compare-Object $a $b -Passthru -IncludeEqual -ExcludeDifferent
    Compare-Object @($a.Keys) @($b.Keys) -Passthru -IncludeEqual -ExcludeDifferent

In [ ]:
# Group-Object

Get-Process | Group-Object Company

In [ ]:
# Group-Object example 1

$data = @'
Name|Number|Email|
Bob| 23| bob.bob@test.com|
Jeff|127|jeff.jeff@test.com|
Jeff|129|jeff.jeff@test.com|
Jessica|126|jessica.jessica@test.com|
Jessica|132|jessica.jessica@test.com|
'@ | ConvertFrom-Csv -Delimiter '|'

$data | Group-Object -Property Name, Email |
        Select @{'Name' = 'Name' ; 'Expression' = { $_.Group[0].Name } },
        @{'Name' = 'Number' ; 'Expression' = { $_.Group.Number } },
        @{'Name' = 'Email' ; 'Expression' = { $_.Group[0].Email } }

In [ ]:
# Group-Object example 2

$data = @(
    [pscustomobject]@{Id='1';Service='Service1';Propertyx=1;Price='5'}
    [pscustomobject]@{Id='1';Service='Service2';Propertyx=1;Price='4'}
    [pscustomobject]@{Id='2';Service='Service1';Propertyx=1;Price='17'}
)

$data |Group-Object Id

$data |Group-Object Id |ForEach-Object {
    [pscustomobject]@{
    Id  = $_.Name
    Sum = $_.Group |Measure-Object Price -Sum |ForEach-Object Sum
    #Sum = $_.Group | Select Price
    }
}

In [ ]:
# Create custom object and property

    $person = [PSCustomObject]@{
        Name = "John"
        Age  = 30
    }

# Add the "Email" property
    $person | Add-Member -MemberType NoteProperty -Name "Email" -Value "john@example.com"

# Display Name and Email
    $person | Select-Object Name, Email

#### Variables ####

In [ ]:
# Variables

    $a, $b = 1, 2
    $a = "test1","test2"

In [ ]:
# Types

    $myVariable = (Get-ChildItem C:\Windows).Name
    $myVariable.GetType() #Array
    $myVariable | Get-Member #String

In [ ]:
# If you had previously statically typed the variable (using something like [int]$a) 
# or if you were in a scope where $a was constrained (like inside a script or function where $a was predefined), 
# then PowerShell could throw an error when you try to reassign it to a different type, such as a hashtable.

    [System.Int32]$a = 10
    $a = "asdf" #error

    [int]$a = 10
    $a = [string]100.1 + [string]9.9
    $intVariable= [double]$intVariable

In [ ]:
# Validation

[validateRange(1,5)][int]$a = 4
[validateLength(0,4)][string]$s = 'abed'

In [ ]:
# Commands for variables

    Test-Path Variable:myVariable

    New-Variable -Name pi -Value 3.14 -Option Constant

    Remove-Variable a
    del Variable:a

In [ ]:
# User input

    $test = Read-Host "Press Enter"
    $test = @( 'MyDesktop', 'MyLaptop', 'YourDesktop', 'Yourlaptop' ) | Out-GridView -PassThru -Title 'Select your favorite computer'

#### Object types ####

##### Strings #####

In [ ]:
# Interpolation

'Строка в "одинарных" кавычках'
#Строка в "одинарных" кавычках

"Строка в 'двойных' кавычках"
#Строка в 'двойных' кавычках

'Строка в ''одинарных кавычках'
#Строка в 'одинарных кавычках

"Строка в ""двойных кавычках"
#Строка в "двойных кавычках

'$a равно $a'
#$a равно $a

"3+2 равно $(3+2)"
#3+2 равно 5

In [ ]:
# Multiline string

$a=@"
1 Первая строка
$(1+1) Вторая строка
Третья строка"
"@

In [ ]:
# Commands

    Select-String '(Error|Setup)' $env:windir\*.log

In [ ]:
# Regular expressions

    "file.doc" -like "f*.doc"

##### DateTime #####

In [ ]:
# DateTime

#Get date per day
    $dl=Get-Date -Year 2021 -Month 03 -Day 08
    $d2=Get-Date -Year 2021 -Month 02 -Day 23

# Substract dates
    ($dl-$d2).Days #12

# Parsing date
    $Date = "18.12.2024"
    $DateTime = [DateTime]::ParseExact($Date, "dd.MM.yyyy", $null)
    $SQLDate = $Datetime.ToString("yyyy-MM-dd")

# Date formats
    (Get-Date).GetDateTimeFormats()

##### Arrays #####

In [ ]:
# Arrays

    $a = 1, 2
    $b = $a

    $a[0] = 3
    $b #3,2

In [ ]:
# Array index

    $a[1..2] #2 3

    $a = 1..5
    $a = $a[0, 1 + 3..($a.length-1)] #1 2 4 5

In [ ]:
# Nested arrays

    $a = @(1, 2)
    $b = @(1, 2)
    $c =@()
    $c += $a, $b #1 2 1 2

    foreach ($obj in $c) {"object"} # object object
    foreach ($obj in $c) {foreach ($o in $obj) {"object"}} # object object object object

##### Hash tables #####

In [ ]:
# Creating, changing, getting values

    $Employee_Numbers = @{"John Doe" = 112233; "Dave Davis" = 223344; "Justine Smith" = 334455}
    $Employee_Numbers += @{"Scott" = 1;}
    $Employee_Numbers.Add("Richard","2")

    $Employee_Numbers.Scott = 2
    $Employee_Numbers.Scott # 2
    $Employee_Numbers["John Doe","Dave Davis"]

In [7]:
#Multiline creation syntax

#Doesn't work if value is an array. 
#; at the end of the lines can be added but are not required

$hash = @{
    'About Arcane Code' = 'http://arcanecode.me'
    ArcaneCode = 'https://arcanecode.com'
    'ArcaneCode RedGate Articles' = 'http://arcanecode.red'
    'ArcaneCode Github Repository' = 'http://arcanerepo.com'}

In [ ]:
# Iteration

    foreach ($h in $hash.GetEnumerator() ) {Write-Host "$($h.Name) : $($h.Value)"}
    foreach ($key in $hash.Keys) { Write-Host "$key : $($hash[$key])" }
    foreach ($h in $hash.Keys) {Write-Host "$h : $($hash.Item($h))"}

In [ ]:
# Comparison

    $a = @{ "one" =  1; "two"= 2 }
    $b = @{ "three" =  3; "two"= 22 }
    Compare-Object @($a.Keys) @($b.Keys) -Passthru -IncludeEqual -ExcludeDifferent

In [ ]:
Hash tables are unordered collections, meaning the order of elements isn't guaranteed

#### Data structures ####

In [ ]:
### Multidimensional/jagged arrays
https://powershellbyexample.dev/post/multidimensional-arrays/

$array = New-Object 'object[,]' 5,8
$array[2,5] = 'Hello'
$array[3,7] = 'World!'
$array

###

$mdarray1 = @()
$mdarray1_counter = 0
$mdarray1 += ,@($mdarray1_counter++, 'Earth',12742)
$mdarray1 += ,@($mdarray1_counter++, 'Mars',6779)
$mdarray1[1][1] #Mars

#In PowerShell, the ,@ operator is called the "array splatting" operator. It is used to expand the elements of an array into the encl
#In the given code, the ,@ operator is used to add the array ($mdarray1_counter++, 'Earth',12742) as a single element to the end of the $mdarray1 array. 
#Without the ,@ operator, $mdarray1 += ($mdarray1_counter++, 'Earth',12742) would add 
#the individual elements of the array to $mdarray1 instead of the entire array as a single element.

In [ ]:
### PSObject

<#$data = @'
Name|Number|Email|
Bob| 23| bob.bob@test.com|
Tom|124|tom.tom@test.com|
Jeff|125|jeff.jeff@test.com|
Jeff|127|jeff.jeff@test.com|
Jeff|129|jeff.jeff@test.com|
Jessica|126|jessica.jessica@test.com|
Jessica|132|jessica.jessica@test.com|
'@ | ConvertFrom-Csv -Delimiter '|'

$row = @'
Name|Number|Email|
Nikita|30|a@a.com|
'@ | ConvertFrom-Csv -Delimiter '|'#>

$data += @([pscustomobject]@{Name="1";Number="2";Email="3"})

$data | Select-Object -ExcludeProperty Name

###

$data += @(
[pscustomobject]@{Id='1';Service='Service1';Propertyx=1;Price='5'}
[pscustomobject]@{Id='1';Service='Service2';Propertyx=1;Price='4'}
[pscustomobject]@{Id='2';Service='Service1';Propertyx=1;Price='17'})
$data[1].id = 3

###

https://www.undocumented-features.com/2019/10/09/creating-an-array-with-header-columns-from-a-string-using-pscustomobject/

[System.Collections.ArrayList]$ArrayWithHeader = @()
$ArrayOfValues = @('a@b.com','b@c.com','c@d.com','d@e.com')
foreach ($obj in $ArrayOfValues) 
{ 
    $val = [pscustomobject]@{'mail'=$obj;'date'=(Get-Date)}
    $ArrayWithHeader.add($val) | Out-Null
}

$ArrayWithHeader

###

$Array0, $Array1 | ForEach-Object {
    [PSCustomObject]@{
        $PSItem[0].Name = $PSItem[0].Value
        $PSItem[1].Name = $PSItem[1].Value
    }
}

In [ ]:
### Datatables

$tempTable = New-Object System.Data.DataTable

###Adding Columns for DataTable###
$tempTable.columns.Add("col1")
$tempTable.columns.Add("col2")
$tempTable.columns.Add("col3")

#Add a row to DataTable

$row = $tempTable.NewRow()
$row[“col1”] = “RandomStringData12”
$row[“col2”] = “RandomStringData22”
$row[“col3”] = "RandomStringData32"
$tempTable.rows.Add($row)

#If you want to modify a table entry you need to select it using Where-Object or the Where() method.
$m = $tempTable.where({$_.title -eq 'Justice League'})

#Then you can modify it. The tricky part to remember is that $m is technically a collection of DataRow objects, even though there is only one.
$m[0].comments = "DC Comics"

#Or I could have modified it like this:
$tempTable.where({$_.title -eq 'Justice League'}).foreach({$_.Comments = "DC Universe"})

#### Flow control ####

##### Iteration #####

In [ ]:
# foreach

    $n = 0
    foreach ($f in dir *.txt) { $n += $f.length }

    $n = 0
    dir *.txt | foreach { $n += length }

##### Loops #####

In [ ]:
# Labeled break statement

    :outerLoop while ($true) {
        while ($true) {
            Write-Host "In inner loop"
            break outerLoop
        }
        Write-Host "This will not be executed"
    }

##### Conditionals #####

In [ ]:
# Switch example 1

$msg = "Error, the action failed"

switch ($msg) {
    { $_ -like "Error*" } { "Action error" }
    { $_ -like "Warning*" } { "Action warning" }
    { $_ -like "Successful*" } { "Action succesfull" }
}

In [ ]:
# Switch example 2

    $a = 3

    switch ($a) {
        1 {"Один"}
        2 {"Два"}
        3 {"Три"; break}
        4 {"Четыре"}
        3 {"Еще раз три"}
    }
    #Три

In [ ]:
# Switch example 3

    switch (3) {
        1 {"1"}
        2 {"2"}
        default {"none"}
    }

In [ ]:
# Switch example 4

    switch -wildcard ('абв') {
        'а*' {"$_ начинается с а"}
        '*в' {"$_ оканчивается на в"}
    }

In [ ]:
# Switch example 5

switch (10) {
    {$_ -gt 5} {"$_ >5"}
    {$_ -lt 20} {"$_ <20"}
    10 {"$_ =10"}
}

In [ ]:
# Switch example 6

switch -wildcard (dir $env:SystemRoot) {
    *.txt {$txt++}
    *.log {$log++} 
}

In [ ]:
# Switch example 7

switch -wildcard -file $env:SystemRoot\KB946627.log {
    *Source:* {$_}
    *Destination:* {$_}
}

#### Functions ####

In [ ]:
# Function example 1

    function MyFunc { Get-Command -Noun Process }

In [ ]:
# Function example 2 - parameters

function Set-ArchiveFilePath{
    [CmdletBinding()]
    [OutputType([string])]
    param(
        [Parameter(Mandatory = $true)]
        [string]$ZipPath,
        [Parameter(Mandatory = $true)]
        [string]$ZipPrefix,
        [Parameter(Mandatory = $true)]
        [datetime]$Date
    )

    Write-Host "this is a function code"

}

In [ ]:
# Function example 3

    function split($items, $splitter) {
        ForEach ($item in $items) {
            $item -split "\$splitter" 
        }
    }

    split 'asd\f' \ #asd, f

In [ ]:
# Function example 4

    function Output-SalesTax ( $Price, [int]$Tax ) { $Price / $Tax }
    
    Output-SalesTax -price 1000 -tax 38
    Output-SalesTax -tax 38 -price 1000 
    Output-SalesTax 1000 38

In [ ]:
# Function example 5

    function Get-UserInfo {
        param (
            [string]$Name,
            [int]$Age,
            [string]$Email
        )

        Write-Host "Name: $Name"
        Write-Host "Age: $Age"
        Write-Host "Email: $Email"
    }

    Get-UserInfo -Name "John Doe" -Age 30 -Email "john.doe@example.com"

In [ ]:
# Function example 6

    function MyFunc ([switch] $recurse) {
        if ($recurse) { "#"}
        else { "Regular function" } }
    MyFunc -recurse #Recursive function

    #Write-host выводит строку на экран, а не в выходной поток, поэтому она не попадет в значение переменной

In [ ]:
Pipeline to a function

Если в функцию могут поступать какие-либо параметры по конвейеру, то в ней
обязательно нужно Определить раздел Process.
Для эффективного
использования конвейеров возвращать значения в расширенных функциях следует
в разделе Process (напомним, что он выполняется каждый раз при получении по
конвейеру нового объекта), а не в разделах Begin или End
В расширенных функциях всегда нужно указывать атрибут [CmdietBindingO ],
в этом случае функция будет вести себя как командлет, в том числе будет поддерживать
флаги -verbose (вывод подробной информации о выполняемых командах)
и -Debug (вывод отладочной информации).

In [ ]:
# Pipeline to a function

    function ProcessFiles {
        param (
            [string]$Prefix
        )
        
        process {
            # Example action: Write the name of each file with the prefix
            Write-Host "$($Prefix): $($_.Name)"
        }
    }
    
    Get-ChildItem | ProcessFiles -Prefix "File"

In [ ]:
# Filter function to be used in a pipeline

    filter FileSizeBelow($size) { if ($_.length -le $size) { $_ } }
    gci C:\Windows | FileSizeBelow 200kb

In [ ]:
When returning data from a function, it is best to save it to a variable in the function
and have just that variable on the last line of the function

In [ ]:
# Lambda / Script-block - 1

    $alert = { "Hello World" }
    & $alert

In [ ]:
Lambda / Script-block - 2

Each time the scriptblock is run; it will dynamically read the current value of the variable.
When a scriptblock is run using the “&” (call) operator, updates to a variable are not reflected in the parent scope.
When a scriptblock is run using the “.” (dot) operator, updates to a variable apply to the current scope.
Begin {}, Process {} and End {} blocks can be added to a scriptblock, just like a function

In [ ]:
# Lambda / Script-block - 3

    $add = { param($a, $b) $a + $b }
    $result = &$add 5 3
    Write-Host "Result: $result"

    $add = { 3 + 5 }
    $result = &$add
    Write-Host "Result: $result"

#### Meta ####

In [ ]:
# Alias

Set-Alias list Get-Location
Set-Alias np c:\windows\notepad.exe
Get-Alias

In [ ]:
# Methods

$methodName = "ToUpper"
$object = "hello"
$object.GetType().GetMethod($methodName).Invoke($object, $null)

$method = [scriptblock]::Create('"hello".ToUpper()')
& $method

In [ ]:
# Parameters

# Parameter from a variable
    $recurse = $false
    Get-ChildItem c:\windows -Recurse:$recurse

# Parameters as hashtable
    $GetWmiObjectParams = @{
        Class = "Win32_LogicalDisk"
        Filter = "DriveType=3"
        ComputerName = "SERVER2”
        }
    Get-WmiObject @GetWmiObjectParams

#### Other ####

In [ ]:
### Working with PSDrives and Providers

    Get-PSDrive
    Get-PSProvider

    Get-ChildItem #dir, ls

    New-PSDrive -Name docs -PSProvider FileSystem -Root C:\Users\

    New-Item

    Remove-Item

In [ ]:
# Self-Update

    Copy-Item -Path $ScriptUpdateLocation -Destination $mypath -Force
    Get-Content $ScriptUpdateLocation -Raw | Set-Content $PSCommandPath
    #$scriptContent | Out-File -FilePath "$PSScriptRoot\Check_SQL_Logins_Fixed.ps1" -Encoding utf8
    Invoke-Expression -Command $scriptContent

In [2]:
# Bonus

( 392, 330, 349, 247, 330, 349, 392, 330 ) | ForEach { [Console]::Beep( $_, 500 ) } # use 32767 for pauses